## Playoff Picture

In [ ]:
# !pip install nflreadpy

In [43]:
import os
import re
from pprint import pprint

import numpy as np
import pandas as pd

import nflreadpy as nfl

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [165]:
folder = 'data/'
filename = folder + 'Official_25-26_Fantasy_Football_Playoff_Edition.csv'
df = pd.read_csv(filename)

new_cols ={
    'Funny Team Name | Your Name': 'Team_Name',
    'Group 1 Quarterbacks': "QB1",
    'Group 2 Quarterbacks': 'QB2', 
    'Group 3 Quarterbacks': 'QB3',
    'Group 4 Quarterbacks': 'QB4', 
    'Group 1 Running Backs': 'RB1',
    'Group 2 Running Backs': 'RB2', 
    'Group 3 Running Backs': 'RB3',
    'Group 4 Running Backs': 'RB4', 
    'Group 1 Wide Receivers': 'WR1',
    'Group 2 Wide Receivers': 'WR2', 
    'Group 3 Wide Receivers': 'WR3',
    'Group 4 Wide Receivers': 'WR4', 
    'Group 5 Wide Receivers': 'WR5',
    'Group 6 Wide Receivers': 'WR6', 
    'Group 1 Tight Ends': 'TE1', 
    'Group 2 Tight Ends': 'TE2',
    'Group 3 Tight Ends': 'TE3', 
    'Group 4 Tight Ends': 'TE4'
}
df = df.rename(columns=new_cols)

pos_cols = ['QB1', 'QB2', 'QB3', 'QB4', 
            'RB1', 'RB2', 'RB3', 'RB4', 
            'WR1', 'WR2', 'WR3', 'WR4', 'WR5', 'WR6', 
            'TE1', 'TE2', 'TE3', 'TE4']

for col in pos_cols:

    # Split the position column into two new columns - player and team
    new_columns = df[col].str.split('|', expand=True)

    # Assign names to the new columns
    new_columns.columns = [f"{col}_Name", f'{col}_Team']

    # Add the new columns to the original DataFrame and drop the old 'Name' column
    df = pd.concat([df, new_columns], axis=1).drop(col, axis=1)

df.head()

,Timestamp,Username,Team_Name,QB1_Name,QB1_Team,QB2_Name,QB2_Team,QB3_Name,QB3_Team,QB4_Name,QB4_Team,RB1_Name,RB1_Team,RB2_Name,RB2_Team,RB3_Name,RB3_Team,RB4_Name,RB4_Team,WR1_Name,WR1_Team,WR2_Name,WR2_Team,WR3_Name,WR3_Team,WR4_Name,WR4_Team,WR5_Name,WR5_Team,WR6_Name,WR6_Team,TE1_Name,TE1_Team,TE2_Name,TE2_Team,TE3_Name,TE3_Team,TE4_Name,TE4_Team
0,2026/01/06 9:44:48 PM EST,mdwheeler0923@gmail.com,Touch of the Downs,Trevor Lawrence,Jacksonville Jaguars,Jalen Hurts,Philadelphia Eagles,Sam Darnold,Seattle Seahawks,Aaron Rodgers,Pittsburgh Steelers,Christian McCaffery,San Francisco 49ers,James Cook,Buffalo Bills,Josh Jacobs,Green Bay Packers,Brian Robinson Jr.,San Francisco 49ers,Davante Adams,Los Angeles Rams,Khalil Shakir,Buffalo Bills,Kayshon Boutte,New England Patriots,DK Metcalf,Pittsburgh Steelers,Joshua Palmer,Buffalo Bills,Calvin Austin III,Pittsburgh Steelers,Dallas Goedert,Philadelphia Eagles,Dalton Kincaid,Buffalo Bills,AJ Barner,Seattle Seahawks,Pat Freiermuth,Pittsburgh Steelers
1,2026/01/07 5:47:10 AM EST,mlabossiere2@gmail.com,Maye Day / Mark LaBossiere,Drake Maye,New England Patriots,Matthew Stafford,Los Angeles Rams,Sam Darnold,Seattle Seahawks,C.J. Stroud,Houston Texans,Kenneth Walker III,Seattle Seahawks,TreVeyon Henderson,New England Patriots,Zach Charbonnet,Seattle Seahawks,Ty Johnson,Buffalo Bills,Puka Nacua,Los Angeles Rams,Stefon Diggs,New England Patriots,Kayshon Boutte,New England Patriots,Rashid Shaheed,Seattle Seahawks,DeMario Douglas,New England Patriots,Tutu Atwell,Los Angeles Rams,Hunter Henry,New England Patriots,Dalton Kincaid,Buffalo Bills,AJ Barner,Seattle Seahawks,Austin Hooper,New England Patriots
2,2026/01/07 7:40:21 AM EST,michaelroyosterman@gmail.com,Mike's legion of boomers,Trevor Lawrence,Jacksonville Jaguars,Jalen Hurts,Philadelphia Eagles,Sam Darnold,Seattle Seahawks,Bryce Young,Carolina Panthers,Kenneth Walker III,Seattle Seahawks,Travis Etienne Jr.,Jacksonville Jaguars,Zach Charbonnet,Seattle Seahawks,Rico Dowdle,Carolina Panthers,Jaxon Smith-Njigba,Seattle Seahawks,Jayden Reed,Green Bay Packers,Pat Bryant,Denver Broncos,Cooper Kupp,Seattle Seahawks,Kyle Williams,New England Patriots,Marvin Mims Jr.,Denver Broncos,Hunter Henry,New England Patriots,Brenton Strange,Jacksonville Jaguars,AJ Barner,Seattle Seahawks,Austin Hooper,New England Patriots
3,2026/01/08 10:52:16 PM EST,jcyork1@gmail.com,I Touchdown There (JC),Drake Maye,New England Patriots,Matthew Stafford,Los Angeles Rams,Sam Darnold,Seattle Seahawks,C.J. Stroud,Houston Texans,Kenneth Walker III,Seattle Seahawks,TreVeyon Henderson,New England Patriots,Josh Jacobs,Green Bay Packers,Rico Dowdle,Carolina Panthers,Jaxon Smith-Njigba,Seattle Seahawks,Stefon Diggs,New England Patriots,DJ Moore,Chicago Bears,DK Metcalf,Pittsburgh Steelers,Brandin Cooks,Buffalo Bills,Christian Kirk,Houston Texans,George Kittle,San Francisco 49ers,Dalton Kincaid,Buffalo Bills,Dalton Schultz,Houston Texans,Oronde Gadsden II,Los Angeles Chargers
4,2026/01/09 3:37:48 PM EST,gatormike2@gmail.com,DUUUUUUUUUUUUUVAL,Trevor Lawrence,Jacksonville Jaguars,Matthew Stafford,Los Angeles Rams,Sam Darnold,Seattle Seahawks,Jordan Love,Green Bay Packers,Kyren Williams,Los Angeles Rams,James Cook,Buffalo Bills,Josh Jacobs,Green Bay Packers,Rico Dowdle,Carolina Panthers,Puka Nacua,Los Angeles Rams,Stefon Diggs,New England Patriots,Rome Odunze,Chicago Bears,Tetairoa McMillan,Carolina Panthers,Keon Coleman,Buffalo Bills,Jaylin Noel,Houston Texans,George Kittle,San Francisco 49ers,Dalton Kincaid,Buffalo Bills,Tyler Higbee,Los Angeles Rams,Oronde Gadsden II,Los Angeles Chargers


##### Reorganize data for Points calculation

In [166]:
team_abbreviations = {  'Buffalo Bills': 'BUF',
                        'Carolina Panthers': 'CAR',
                        'Chicago Bears': 'CHI',
                        'Denver Broncos': 'DEN',
                        'Green Bay Packers': 'GB',
                        'Houston Texans': 'HOU',
                        'Jacksonville Jaguars': 'JAX',
                        'Los Angeles Chargers': 'LAC',
                        'Los Angeles Rams': 'LAR',
                        'New England Patriots': 'NE',
                        'Philadelphia Eagles': 'PHI',
                        'Pittsburgh Steelers': 'PIT',
                        'San Francisco 49ers': 'SF',
                        'Seattle Seahawks': 'SEA'}

player_name_fixes = {
   'Brian Robinson Jr.': "Brian Robinson",
    'Christian McCaffery': 'Christian McCaffrey',
    'Joshua Palmer': 'Josh Palmer',  # Ruled out due to ankle injury
    'Travis Etienne Jr.': "Travis Etienne",
    'Dontayvion Wicks': None,  #  Ruled out due to concussion
    'Troy Franklin': None,   # Hasn't played yet this post-season
    'Tutu Atwell': None,  # Hasn't played yet this post-season
    'Ty Johnson': None  # Ruled out due to injury
}

In [167]:
## Update teams to abbreviations and fix player names
for i, row in df.iterrows():
    for col in pos_cols:
        player_name = row[f'{col}_Name'].strip()
        player_team = row[f'{col}_Team'].strip()
        
        # Fix player names if needed
        if player_name in player_name_fixes:
            fixed_name = player_name_fixes[player_name]
            if fixed_name is None:
                continue  # Skip players who are ruled out or haven't played
            df.at[i, f'{col}_Name'] = fixed_name
        
        # Update team to abbreviation if needed
        if player_team in team_abbreviations:
            df.at[i, f'{col}_Team'] = team_abbreviations[player_team]

In [168]:
# Use Team_Name as the manager name (change if you want Username instead)
manager_col = "Team_Name"
rows = []

for _, row in df.iterrows():
    manager = row[manager_col]

    # find all columns that end with _Name
    for col in df.columns:
        if col.endswith("_Name"):
            player = row[col].strip()

            # matching team column
            team_col = col.replace("_Name", "_Team")
            team = row.get(team_col)

            # skip empty cells
            if pd.notna(player) and pd.notna(team):
                rows.append({
                    "Manager_Name": manager,
                    "Player_Name": player,
                    "NFL_Team": team
                })

final_df = pd.DataFrame(rows)
final_df.head(2)

,Manager_Name,Player_Name,NFL_Team
0,Touch of the Downs,Trevor Lawrence,JAX
1,Touch of the Downs,Jalen Hurts,PHI


NFL Stats   
---

In [ ]:
# Load current season play-by-play data
pbp = nfl.load_pbp().to_pandas()
# pbp_pass = pbp[pbp.play_type == 'pass']
# pbp.play_type.value_counts()

# Load player game-level stats for multiple seasons
player_stats = nfl.load_player_stats(summary_level='week').to_pandas()   # using 'post' aggregates all post season stats

# Load all available team level stats
team_stats = nfl.load_team_stats(summary_level='week').to_pandas()


####  -----  ####
# sorted(player_stats1.columns)
# player_stats.query("week>18 and player_display_name=='Trevor Lawrence'")[sorted(player_stats.columns)]

#### Establishing Teams by Round

In [169]:
## Adding the 4 weeks of Playoffs and Filtering to only teams playing 
wc_teams = ['SEA', 'DEN']
final_df['Wildcard'] = final_df.NFL_Team.apply(lambda t: False if t in wc_teams else True)

# Divisional 
div_teams = ['SEA', 'DEN', 'SF', 'LAR', 'CHI', 'BUF', 'HOU', 'NE']
final_df['Division'] = final_df.NFL_Team.isin(div_teams)

# Conference
conf_teams = ['SEA', 'DEN', 'LAR', 'NE']
final_df['Conference'] = final_df.NFL_Team.isin(conf_teams)

# Super Bowl
# sb_teams = []
# final_df['Super Bowl'] = final_df.NFL_Team.isin(sb_teams)

final_df.to_csv('data/managers_games.csv')
final_df.sample(5)

,Manager_Name,Player_Name,NFL_Team,Wildcard,Division,Conference
140,Duvallll,Dallas Goedert,PHI,True,False,False
224,HoosierDaddy / Greg McBride,Puka Nacua,LAR,True,True,True
314,Sleepless in South Beach,Jaxon Smith-Njigba,SEA,False,True,True
142,Duvallll,Tyler Higbee,LAR,True,True,True
22,Maye Day / Mark LaBossiere,Kenneth Walker III,SEA,False,True,True


##### Adding in Post-Season Points

In [154]:
# Define passing points function

def passing_points(w, name, qb_print=False):
    if player_stats.query("week==@w & player_display_name==@name").shape[0] == 0:
        return 0
    else:
        pass_yards = player_stats.query("week==@w & player_display_name==@name")['passing_yards'].values[0]
        pass_td = player_stats.query("week==@w & player_display_name==@name")['passing_tds'].values[0]
        int_ = player_stats.query("week==@w & player_display_name==@name")['passing_interceptions'].values[0]
        two_pt = player_stats.query("week==@w & player_display_name==@name")['passing_2pt_conversions'].values[0]
        sack = player_stats.query("week==@w & player_display_name==@name")['sacks_suffered'].values[0]
        fumble = player_stats.query("week==@w & player_display_name==@name")['sack_fumbles'].values[0]
        # td40yd = sum(player_stats.query("week==@w & player_display_name==@name & series_result=='Touchdown' & td_team.notnull()")['passing_yards'] >= 40)
        # td50yd = sum(player_stats.query("week==@w & player_display_name==@name & series_result=='Touchdown' & td_team.notnull()")['passing_yards'] >= 50)
        if pass_yards > 400:
            bonus = 3
        elif pass_yards > 300:
            bonus = 2
        else:
            bonus=0
        total = (.04*pass_yards) + (6*pass_td) + (-2*int_) + (2*two_pt) + (-1*sack) + (-2*fumble) + bonus    ## + (2*td50yd) + td40yd 
        if qb_print == True:
            print(f"Passing Yards: {pass_yards} \nPassing TDs: {pass_td} \nInterceptions: {int_} \nTwo Point Conversions: {two_pt}\
                \nSacks: {sack} \nFumbles: {fumble} \nBonus: {bonus} \nTotal: {round(total, 6)}")   # \n40+ Yard TDs: {td40yd} \n50+ Yard TDs: {td50yd} 
        
        return round(total, 4)

In [153]:
# Define Receiving Points function

def receiving_points(w, name, rec_print=False):
    if player_stats.query("week==@w & player_display_name==@name").shape[0] == 0:
        return 0
    else:
        rec_yards = player_stats.query("week==@w & player_display_name==@name")['receiving_yards'].values[0]
        rec_td = player_stats.query("week==@w & player_display_name==@name")['receiving_tds'].values[0]
        fumble = player_stats.query("week==@w & player_display_name==@name")['receiving_fumbles'].values[0]
        two_pt = player_stats.query("week==@w & player_display_name==@name")['receiving_2pt_conversions'].values[0]  
        recptn = player_stats.query("week==@w & player_display_name==@name")['receptions'].values[0]
        # td40yd = pbp_pass.query("week==@w & receiver==@name & series_result=='Touchdown' & td_team.notnull() & receiving_yards>=40 & receiving_yards<50")['receiving_yards'].count()
        # td50yd = pbp_pass.query("week==@w & receiver==@name & series_result=='Touchdown' & td_team.notnull() & receiving_yards>=50")['receiving_yards'].count()
        if rec_yards > 200:
            bonus = 3
        elif rec_yards > 100:
            bonus = 2
        else:
            bonus=0
        total = (.1*rec_yards) + (6*rec_td) + (-2*fumble) + (2*two_pt)+ bonus + recptn    #  + (2*td50yd) + td40yd 
        if rec_print == True:
            print(f"Receiving Yards: {rec_yards} \nReceiving TDs: {rec_td} \nFumbles: {fumble} \nTwo Point Conversions: {two_pt}\
                \nBonus: {bonus} \nReceptions: {recptn} \nTotal: {total}")      # \n40+ Yard TDs: {td40yd} \n50+ Yard TDs: {td50yd} 
        
        return round(total, 4)

In [152]:
# Define Rushing points function

def rushing_points(w, name, rush_print=False):
    if player_stats.query("week==@w & player_display_name==@name").shape[0] == 0:
        return 0
    else:
        rush_yards = player_stats.query("week==@w & player_display_name==@name")['rushing_yards'].values[0]
        rush_td = player_stats.query("week==@w & player_display_name==@name")['rushing_tds'].values[0]
        fumble = player_stats.query("week==@w & player_display_name==@name")['rushing_fumbles'].values[0]
        two_pt = player_stats.query("week==@w & player_display_name==@name")['rushing_2pt_conversions'].values[0] 
        # td40yd = pbp_run.query("week==@w & rusher==@name & series_result=='Touchdown' & td_team.notnull() & rushing_yards>=40 & rushing_yards<50")['rushing_yards'].count()
        # td50yd = pbp_run.query("week==@w & rusher==@name & series_result=='Touchdown' & td_team.notnull() & rushing_yards>=50")['rushing_yards'].count()
        if rush_yards > 200:
            bonus = 3
        elif rush_yards > 100:
            bonus = 2
        else:
            bonus=0
        total = (.1*rush_yards) + (6*rush_td) + (-2*fumble) + (2*two_pt) + bonus   ##  + (2*td50yd) + td40yd
        if rush_print == True:
            print(f"Rushing Yards: {rush_yards} \nRushing TDs: {rush_td} \nFumbles: {fumble} \nTwo Point Conversions: {two_pt}\
                \nBonus: {bonus} \nTotal: {total}")  ##  \n40+ Yard TDs: {td40yd} \n50+ Yard TDs: {td50yd}
        
        return round(total, 4)

In [170]:
df = final_df.copy()

# Apply passing points to games played

df['Passing_wc'] = df.Player_Name.apply(lambda x: passing_points(19, x))
df['Passing_div'] = df.Player_Name.apply(lambda x: passing_points(20, x))
# df['Passing_conf'] = df.Player_Name.apply(lambda x: passing_points(21, x))
# df['Passing_sb'] = df.Player_Name.apply(lambda x: passing_points(22, x))  # this may be 22

pass_cols = [c for c in df.columns if "Passing" in c]
df['Total_passing'] = df[pass_cols].sum(axis=1)

# df.sort_values('Total_passing', ascending=False).sample(3)

In [171]:
# Apply receiving points to games played

df['Receiving_wc'] = df.Player_Name.apply(lambda x: receiving_points(19, x))
df['Receiving_div'] = df.Player_Name.apply(lambda x: receiving_points(20, x))
# df['Receiving_conf'] = df.Player_Name.apply(lambda x: receiving_points(21, x))
# df['Receiving_sb'] = df.Player_Name.apply(lambda x: receiving_points(22, x))  # this may be 22

rec_cols = [c for c in df.columns if "Receiving" in c]
df['Total_receiving'] = df[rec_cols].sum(axis=1)

# df.sort_values('Total_receiving', ascending=False).sample(3)

In [172]:
# Apply rushing points to games played

df['Rushing_wc'] = df.Player_Name.apply(lambda x: rushing_points(19, x))
df['Rushing_div'] = df.Player_Name.apply(lambda x: rushing_points(20, x))
# df['Rushing_conf'] = df.Player_Name.apply(lambda x: rushing_points(21, x))
# df['Rushing_sb'] = df.Player_Name.apply(lambda x: rushing_points(22, x))  # this may be 22

rush_cols = [c for c in df.columns if "Rushing" in c]
df['Total_rushing'] = df[rush_cols].sum(axis=1)

# df.sort_values('Total_rushing', ascending=False).sample(5)

##### Points-by-Week

In [173]:
wc_cols = [c for c in df.columns if "wc" in c]
div_cols = [c for c in df.columns if "div" in c]
# conf_cols = [c for c in df.columns if "conf" in c]
# sb_cols = [c for c in df.columns if "sb" in c]

df['Total_WC'] = df[wc_cols].sum(axis=1)
df['Total_Div'] = df[div_cols].sum(axis=1)
# df['Total_Conf'] = df[conf_cols].sum(axis=1)    
# df['Total_SB'] = df[sb_cols].sum(axis=1)

df['Total'] = df['Total_WC'] + df['Total_Div']   #+ df['Total_Conf'] + df['Total_SB']

total_points_df = df[['Manager_Name', 'Player_Name', 'NFL_Team', 'Total_WC', 'Total_Div', 'Total']]   # 'Total_Conf', 'Total_SB', 

dedup_df = total_points_df.sort_values('Player_Name').drop_duplicates('Player_Name', keep='first').reset_index(drop=True)
dedup_df.to_csv('data/total_points.csv')
dedup_df

,Manager_Name,Player_Name,NFL_Team,Total_WC,Total_Div,Total
0,Spencer,A.J. Brown,PHI,5.50,0.00,5.50
1,Maye Day / Mark LaBossiere,AJ Barner,SEA,0.00,0.20,0.20
2,Duvallll,Aaron Rodgers,PIT,-4.16,0.00,-4.16
3,HoosierDaddy / Greg McBride,Austin Hooper,NE,0.00,2.40,2.40
4,Ricky Bobby,Bhayshul Tuten,JAX,5.10,0.00,5.10
5,KaiBearMom/Gigi,Bo Nix,DEN,0.00,30.06,30.06
6,Wide Right Redemption - Tanja,Brandin Cooks,BUF,8.80,4.00,12.80
7,Spencer,Brenton Strange,JAX,2.90,0.00,2.90
8,Touch of the Downs,Brian Robinson,SF,0.00,1.70,1.70
9,Sleepless in South Beach,Brian Thomas Jr.,JAX,10.10,0.00,10.10


In [ ]:
(dedup_df.columns)

Index(['Manager_Name', 'Player_Name', 'NFL_Team', 'Total_WC', 'Total_Div',
       'Total'],
      dtype='object')

#### Create Scoreboard .csv

In [157]:
# Look at just top scorers
tot_cols = [c for c in dedup_df.columns if "Total" in c]

points_df = dedup_df[['Manager_Name', *tot_cols]].copy()
points_df['Total_points'] = dedup_df[tot_cols].sum(axis=1)
points_df

,Manager_Name,Total_WC,Total_Div,Total,Total_points
0,Spencer,5.50,0.00,5.50,11.00
1,Maye Day / Mark LaBossiere,0.00,0.20,0.20,0.40
2,Duvallll,-4.16,0.00,-4.16,-8.32
3,HoosierDaddy / Greg McBride,0.00,2.40,2.40,4.80
4,Ricky Bobby,5.10,0.00,5.10,10.20
5,KaiBearMom/Gigi,0.00,30.06,30.06,60.12
6,Wide Right Redemption - Tanja,8.80,4.00,12.80,25.60
7,Spencer,2.90,0.00,2.90,5.80
8,Touch of the Downs,0.00,1.70,1.70,3.40
9,Sleepless in South Beach,10.10,0.00,10.10,20.20


In [164]:
points_df.groupby(['Manager_Name']).sum('Total_points').sort_values('Total_points', ascending=False)

,Total_WC,Total_Div,Total,Total_points
Manager_Name,,,,
The Cannoli Fillers,98.64,101.68,200.32,400.64
McSrq,115.96,48.62,164.58,329.16
Fly Eagles Fly,102.02,52.06,154.08,308.16
KaiBearMom/Gigi,64.72,56.66,121.38,242.76
Dub season,63.50,9.60,73.10,146.20
Bye Felicia,24.40,48.40,72.80,145.60
Duvallll,34.40,31.72,66.12,132.24
HoosierDaddy / Greg McBride,38.70,21.80,60.50,121.00
Wide Right Redemption - Tanja,22.80,34.00,56.80,113.60
